In [1]:
import torch
from tqdm import tqdm

# Image-related utilities
from torchvision.io import decode_image, read_image
from torchvision.transforms import ToTensor
from torchvision import transforms
from PIL import Image

# Import models
from torchvision.models import vgg19, VGG19_Weights
from torchvision.models import vgg16, VGG16_Weights
from torchvision.models import vgg11, VGG11_Weights

# Dataset
from torchvision.datasets import Imagenette, ImageFolder

# Plotting utility
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Read imagenette data into data loader
#Process image
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to VGG19 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Same normalization as ImageNet
])
imagewoof_data = ImageFolder(root='/home/yi/Downloads/imagewoof2/val', transform=transform)
data_loader = torch.utils.data.DataLoader(imagewoof_data, batch_size=1, shuffle=False)

In [3]:
# Retrained model
# PATH = 'vgg19_imagewoof.pth'
# model = vgg19()
# PATH = "vgg16_Imagewoof.pth"
# model = vgg16()
PATH = "vgg11_Imagewoof.pth"
model = vgg11()

model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=10)
model.load_state_dict(torch.load(PATH, weights_only=True))
model = model.to(device)
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
img_path = []
true_label = []
predicted_label = []
confidence = []
true_confidence = []
correctness = []

image_paths = [data_loader.dataset.samples[i][0] for i in range(len(data_loader.dataset))]

with torch.no_grad():
    for idx, (img, label) in enumerate(tqdm(data_loader, desc="Evaluating", unit="images")):
        # Retrieve image
        img = img.to(device)
        label = label.to(device)
        
        # Prediction
        prediction = model(img).squeeze(0).softmax(0)
        class_id = prediction.argmax().item()
        score = prediction[class_id].item()
        true_score = prediction[label.item()].item()
        
        # Correct?
        #category_name = weights.meta["categories"][class_id]
        
        # Extract class names from the dataset
        id_to_class = {v: k for k, v in imagewoof_data.class_to_idx.items()}  # Reverse mapping
        # Get predicted category name
        category_name = id_to_class[class_id]
        # Check if prediction is correct
        correct = 'y' if category_name == id_to_class[label.item()] else 'n'
        
        # Log
        img_path.append(image_paths[idx])
        true_label.append(id_to_class[label.item()])
        predicted_label.append(category_name)
        confidence.append(score)
        true_confidence.append(true_score)
        correctness.append(correct)

# Dictionary of logs
dict = {'image': img_path, 'true label': true_label, 'predicted label': predicted_label,
        'confidence score': confidence, 'true confidence score': true_confidence, 'correct': correctness}
df = pd.DataFrame(dict)
# df.to_csv('vgg19_results_retrained_imagewoof.csv')
# df.to_csv('vgg16_results_retrained_imagewoof.csv')
df.to_csv('vgg11_results_retrained_imagewoof.csv')

Evaluating: 100%|███████████████████████| 3929/3929 [00:50<00:00, 77.37images/s]


In [5]:
# Print evaluation accuracy
print('Evaluation accuracy: ', round(len(df[df['correct'] == 'y']) / len(df), 4))

Evaluation accuracy:  0.8962
